In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
import numpy as np

## Load Pickles of checkpoint charges per byte

In [3]:
from os import getcwd, path, pardir
import pickle
from math import ceil

In [4]:
run_experiments = False

Experiment AES

In [5]:
if run_experiments:
    from Security.AES.experiment_aes import experiment_aes
    experiment_aes(path.join(getcwd(), "Security", "AES"))

encrypt_charge = pickle.load(open(path.join(getcwd(), "Security", "AES", "aes_encrypt_charge.p"), "rb"))
decrypt_charge = pickle.load(open(path.join(getcwd(), "Security", "AES", "aes_decrypt_charge.p"), "rb"))

Experiment AES Flash

In [6]:
if run_experiments:
    from Security.AES_Flash.experiment_aes_flash import experiment_aes_flash
    experiment_aes_flash(path.join(getcwd(), "Security", "AES_Flash"))

aes_encrypt_charge = pickle.load(open(path.join(getcwd(), "Security", "AES_Flash", "aes_flash_encrypt_charge.p"), "rb"))
aes_decrypt_charge = pickle.load(open(path.join(getcwd(), "Security", "AES_Flash", "aes_flash_decrypt_charge.p"), "rb"))
flash_write_charge = pickle.load(open(path.join(getcwd(), "Security", "AES_Flash", "aes_flash_write_charge.p"), "rb"))
flash_read_charge = pickle.load(open(path.join(getcwd(), "Security", "AES_Flash", "aes_flash_read_charge.p"), "rb"))

Experiment TrustZone

In [7]:
if run_experiments:
    from Security.TrustZone.experiment_trustzone import experiment_trustzone
    experiment_trustzone(path.join(getcwd(), "Security", "TrustZone"))

nsc_store_charge = pickle.load(open(path.join(getcwd(), "Security", "TrustZone", "trustzone_store_charge.p"), "rb"))
nsc_load_charge = pickle.load(open(path.join(getcwd(), "Security", "TrustZone", "trustzone_load_charge.p"), "rb"))

Experiment TrustZone Flash

In [8]:
if run_experiments:
    from Security.TrustZone_Flash.experiment_trustzone_flash import experiment_trustzone_flash
    experiment_trustzone_flash(path.join(getcwd(), "Security", "TrustZone_Flash"))

flash_nsc_store_charge = pickle.load(open(path.join(getcwd(), "Security", "TrustZone_Flash", "trustzone_flash_store_charge.p"), "rb"))
flash_nsc_load_charge = pickle.load(open(path.join(getcwd(), "Security", "TrustZone_Flash", "trustzone_flash_load_charge.p"), "rb"))

### Smoothen?

Make lookup function

In [9]:
AES_BLOCK_SIZE = 16
def get_security_charge(security_type, number_of_bytes):
    if security_type is "AES":
        index = ceil(number_of_bytes/AES_BLOCK_SIZE)
        return encrypt_charge[index] + decrypt_charge[index]
    elif security_type is "AES_Flash":
        index = ceil(number_of_bytes/AES_BLOCK_SIZE)
        return aes_encrypt_charge[index] + flash_write_charge[index] + flash_read_charge[index] + aes_decrypt_charge[index]
    elif security_type is "TrustZone":
        return nsc_store_charge[number_of_bytes] + nsc_store_charge[number_of_bytes]
    elif security_type is "TrustZone_Flash":
        return flash_nsc_store_charge[number_of_bytes] + flash_nsc_load_charge[number_of_bytes]
    else:
        return 0 # No security, so 0 C

In [10]:
len(nsc_store_charge)
#16*880

6492

In [160]:
security_types = ["None", "AES", "AES_Flash", "TrustZone", "TrustZone_Flash"]
# security_types = ["None", "AES_Flash", "TruseZone_Flash"]
application_names = ["Kalman", "AR", "Dijkstra", "Sorting 1024", "Bit Counting", "FFT 128"]

In [184]:
# app_charge_scale = 1e-4
application_charges = np.array([0.0001, 1e-10, 1e-10, 0.00001984909516280825944492365031646130546505446545779705047607421875, 0.0000066193645743728758915470737622666064225995796732604503631591796875, 0.00098674225419147416384080262474753908463753759860992431640625]) # TODO
# app_number_of_bytes_scale = 1000
application_number_of_bytes = np.array([234, 1, 1, 1544 + 2**10, 90, 1975+24*128]) # TODO

In [185]:
base_charge_per_app = dict(zip(application_names, application_charges))
number_of_bytes_per_app = dict(zip(application_names, application_number_of_bytes))

In [186]:
def get_total_charge(application_name, security_type):
    print(application_name, security_type, f"base: {base_charge_per_app[application_name]}, sec: {get_security_charge(security_type, number_of_bytes_per_app[application_name])}")
    return base_charge_per_app[application_name] + get_security_charge(security_type, number_of_bytes_per_app[application_name])

# Bar chart

In [174]:
import numpy as np
import matplotlib.pyplot as plt

In [175]:
# create plot
fig, ax = plt.subplots(figsize=(10,9))
index = np.arange(len(application_names))
bar_width = 0.18
opacity = 0.8

# ax.grid()

colors = ["blue", "red", "green", "cyan", "magenta", "yellow", "black", "white"]

rects = []
for i,security_type in enumerate(security_types):
    rects += ax.bar(index + i*bar_width, [get_total_charge(application_name, security_type) for application_name in application_names], bar_width, label=security_type, color=colors[i], alpha=opacity)
 
ax.set_xlabel('Application')
ax.set_ylabel('Charge [C]')
ax.set_title('Charge per Application')
fig.canvas.set_window_title('Charge per Application')
ax.set_xticks(index + bar_width)
ax.set_xticklabels(application_names)
ax.legend()
 
fig.tight_layout()
fig.show()

Kalman None base: 0.0001, sec: 0
AR None base: 0.0, sec: 0
Dijkstra None base: 0.0, sec: 0
Sorting 1024 None base: 1.984909516280826e-05, sec: 0
Bit Counting None base: 6.619364574372876e-06, sec: 0
FFT 128 None base: 0.0009867422541914742, sec: 0
Kalman AES base: 0.0001, sec: 2.134558538557974e-05
AR AES base: 0.0, sec: 3.0560544495826275e-06
Dijkstra AES base: 0.0, sec: 3.0560544495826275e-06
Sorting 1024 AES base: 1.984909516280826e-05, sec: 0.000173811552841254
Bit Counting AES base: 6.619364574372876e-06, sec: 1.1982284226705715e-05
FFT 128 AES base: 0.0009867422541914742, sec: 0.0003366292979513878
Kalman AES_Flash base: 0.0001, sec: 3.439272529308316e-05
AR AES_Flash base: 0.0, sec: 1.1511941748189174e-05
Dijkstra AES_Flash base: 0.0, sec: 1.1511941748189174e-05
Sorting 1024 AES_Flash base: 1.984909516280826e-05, sec: 0.00024787029089482144
Bit Counting AES_Flash base: 6.619364574372876e-06, sec: 1.8847149752568023e-05
FFT 128 AES_Flash base: 0.0009867422541914742, sec: 0.000472

# Number of Checkpoints

In [176]:
security_types = ["None", "AES_Flash", "TrustZone_Flash"]

In [177]:
min_cap_size_over_sec_charge = 10
number_of_points = 1e5
iterations = 100
min_cap_factor = 10
min_step_size = 1e-7

In [187]:
opacity = 0.8
colors = ["blue", "red", "green", "cyan", "magenta", "yellow", "black", "white"]

figs = {}
axs = {}
for application_name in application_names:
# for application_name in ["Sorting 1024", "FFT 128"]:
    fig, ax = plt.subplots(figsize=(10,9))
    figs[application_name] = fig
    axs[application_name] = ax
    fig.canvas.set_window_title(application_name)
    for i,security_type in enumerate(security_types):
        security_charge = get_security_charge(security_type, number_of_bytes_per_app[application_name])
        base_charge = base_charge_per_app[application_name]
        capacitor_sizes = np.arange(base_charge/min_cap_factor,base_charge*iterations, (base_charge*iterations-base_charge/min_cap_factor)/number_of_points)
        capacitor_sizes = capacitor_sizes[capacitor_sizes>security_charge]
        y = base_charge/(capacitor_sizes - security_charge)
#         ax.plot(capacitor_sizes, base_charge/(capacitor_sizes - security_charge)/(base_charge/capacitor_sizes), label=security_type, color=colors[i], alpha=opacity)
        ax.plot(capacitor_sizes, y, label=security_type, color=colors[i], alpha=opacity)
        ax.plot(capacitor_sizes, np.floor(y), color=colors[i], alpha=opacity/2)
#         ax.plot(capacitor_sizes, np.floor(base_charge/(capacitor_sizes - security_charge)), '.', label=security_type, color=colors[i], alpha=opacity/2)
#         ax.plot(capacitor_sizes, base_charge/(capacitor_sizes - security_charge), label=security_type, color=colors[i], alpha=opacity)
    ax.plot([base_charge/min_cap_factor, base_charge*iterations], [1,1])
    ax.set_xscale('log')
#     ax.set_yscale('log')
    ax.set_ylim(-1,10)
    ax.set_xlabel('Capacitor Size [C]')
    ax.set_ylabel('Number of Checkpoints')
    ax.set_title(f'Number of Checkpoints per Security Type {application_name}')
    fig.canvas.set_window_title(f"Number of Checkpoints {application_name}")
    ax.legend()
    fig.tight_layout()
    fig.show()

In [ ]:
# opacity = 0.8
# colors = ["blue", "red", "green", "cyan", "magenta", "yellow", "black", "white"]

# figs = {}
# axs = {}
# # for application_name in application_names:
# for application_name in ["Sorting 1024", "FFT 128"]:
#     fig, ax = plt.subplots(figsize=(10,9))
#     figs[application_name] = fig
#     axs[application_name] = ax
#     fig.canvas.set_window_title(application_name)
#     security_charge_aes = get_security_charge("AES_Flash", number_of_bytes_per_app[application_name])
#     security_charge_trustzone = get_security_charge("TrustZone_Flash", number_of_bytes_per_app[application_name])
#     base_charge = base_charge_per_app[application_name]
#     capacitor_sizes = np.arange(base_charge/min_cap_factor,base_charge*iterations, (base_charge*iterations-base_charge/min_cap_factor)/number_of_points)
#     capacitor_sizes = capacitor_sizes[capacitor_sizes>security_charge_aes]
# #         ax.plot(capacitor_sizes, base_charge/(capacitor_sizes - security_charge)/(base_charge/capacitor_sizes), label=security_type, color=colors[i], alpha=opacity)
#     y = base_charge/(capacitor_sizes - security_charge_aes)/(base_charge/(capacitor_sizes - security_charge_trustzone))
#     ax.plot(capacitor_sizes, y, label=security_type, color=colors[i], alpha=opacity)
#     ax.plot(capacitor_sizes, np.floor(y), color=colors[i], alpha=opacity/2)
# #         ax.plot(capacitor_sizes, np.floor(base_charge/(capacitor_sizes - security_charge)), '.', label=security_type, color=colors[i], alpha=opacity/2)
# #         ax.plot(capacitor_sizes, base_charge/(capacitor_sizes - security_charge), label=security_type, color=colors[i], alpha=opacity)
#     ax.plot([base_charge/min_cap_factor, base_charge*iterations], [1,1])
#     ax.set_xscale('log')
# #     ax.set_yscale('log')
#     ax.set_ylim(-1,10)
#     ax.set_xlabel('Capacitor Size [C]')
#     ax.set_ylabel('Number of Checkpoints')
#     ax.set_title(f'Number of Checkpoints per Security Type {application_name}')
#     fig.canvas.set_window_title(f"Number of Checkpoints {application_name}")
#     ax.legend()
#     fig.tight_layout()
#     fig.show()

## Scales

In [ ]:
print("Cap Size:")
print(f"\tmin: {min(capacitor_sizes)}, max: {max(capacitor_sizes)}")

In [179]:
print("Base Charge:")
for application_name in application_names:
    print(f"\t{application_name}: {base_charge_per_app[application_name]}")

Base Charge:
	Kalman: 0.0001
	AR: 0.0
	Dijkstra: 0.0
	Sorting 1024: 1.984909516280826e-05
	Bit Counting: 6.619364574372876e-06
	FFT 128: 0.0009867422541914742


In [22]:
print("Security Charge:")
for security_type in security_types:
    print(f"\t{security_type} min: {get_security_charge(security_type, min(number_of_bytes_per_app.values()))},  max: {get_security_charge(security_type, max(number_of_bytes_per_app.values()))}")
# number_of_bytes_per_app = dict(zip(application_names, application_number_of_bytes))

Security Charge:
	None min: 0,  max: 0
	AES_Flash min: 1.3821473868907638e-05,  max: 0.00047226733670911465
	TrustZone_Flash min: 4.708487521232661e-06,  max: 0.00012970356681269365


In [23]:
base_charge_per_app

{'Kalman': 0.0001,
 'AR': 0.0002,
 'Dijkstra': 0.0003,
 'Sorting 1024': 1.984909516280826e-05,
 'Bit Count': 0.0004,
 'FFT 128': 0.000988899251949373}

In [24]:
min(number_of_bytes_per_app.values())

32

In [25]:
get_security_charge("None", min(number_of_bytes_per_app.values()))

0

In [26]:
get_security_charge("TrustZone_Flash", 4000)

0.0001048538910494544

In [27]:
1544 + 2**10

2568

In [28]:
2**13

8192

In [29]:
0x37b7

14263

In [30]:
0x1fb7

8119

In [31]:
0x1fb7+256*24

14263

In [32]:
620*16

9920

In [33]:
0x1fb7-256*24

1975

In [34]:
1975+24*512

14263

In [35]:
1975+24*128

5047

In [36]:
print(f"{1975+24*128:x}")

13b7


In [37]:
print(f"{1975+24*0:x}")

7b7


## Plot stuff

In [38]:
# fig = plt.figure(figsize=(10, 6))
# ax = fig.add_subplot(1, 1, 1)
# ax.set_xlabel("Capacitor Size [C]")
# ax.set_ylabel("Number of Checkpoints")

Text(0, 0.5, 'Number of Checkpoints')

In [39]:
# capacitor_sizes = range(1,101)
# def get_number_of_checkpoints(application_number):
#     return [application_charges[application_number]/(capacitor_size - get_security_charge("trustzone_flash", application_number_of_bytes[application_number])) for capacitor_size in capacitor_sizes]

In [40]:
# for application_number in range(5):
#     ax.plot(capacitor_sizes, get_number_of_checkpoints(application_number), label=application_names[application_number])
# ax.legend()
# # ax.set_yscale('log')
# # ax.set_xscale('log')
# # ax.invert_xaxis()

In [41]:
# fig2 = plt.figure(figsize=(10, 6))
# ax2 = fig2.add_subplot(111, projection='3d')

# ax2.set_xlabel("Capacitor Size [C]")
# ax2.set_ylabel("Number of Bytes")
# ax2.set_zlabel("Number of Checkpoints")

In [42]:
# capacitor_sizes = range(1,1001)
# application_charge = 50
# number_of_bytes = range(1,1001)

In [43]:
# def get_number_of_checkpoints(application_charge, capacitor_size, number_of_bytes, security_type="trustzone_flash"):
#     return application_charge/(capacitor_size - get_security_charge(security_type, number_of_bytes))

In [44]:
# X, Y = np.meshgrid(capacitor_sizes, number_of_bytes)
# Z = np.zeros(shape=(1000,1000))
# for i,capacitor_size in enumerate(capacitor_sizes):
#     for j,number_of_byte in enumerate(number_of_bytes):
#         Z[i,j] = get_number_of_checkpoints(application_charge, capacitor_size, number_of_byte)
# #Z = np.array([np.array([get_number_of_checkpoints(capacitor_size,application_charge) for application_charge in application_charges]) for  capacitor_size in capacitor_sizes])
# Z.shape

In [45]:
# surf = ax2.plot_surface(X, Y, Z)